# Variáveis Instrumentais

Vamos explorar o uso de [variáveis instrumentais](https://en.wikipedia.org/wiki/Instrumental_variables_estimation) replicando alguns resultados apresentados no artigo de Joshua D. Angrist e William N. Evans, "Children and Their Parents' Labor Supply: Evidence from Exogenous Variation in Family Size", *The American Economic Review* Vol. 88, No. 3, 1998, pp. 450-477.

### O Estudo
Angrist e Evans tiveram interesse em estudar o efeito do número de filhos na oferta de trabalho ([labor supply](https://en.wikipedia.org/wiki/Labour_supply)) das mães. Ou seja, a variável fertilidade pode impactar no número de horas que as mães podem (ou estão dispostas a) trabalhar, restringindo as mulheres no desenvolvimento de suas carreiras. De fato, diversas investigações mostram correlação negativa entre número de filhos e oferta de trabalho, mas a intrepretação desta correlação não é muito clara. Relações de causa e efeito são complicadas, pois há fortes indicios para acreditarmos que fertilidade e oferta de trabalho são [variáveis endógenas](https://en.wikipedia.org/wiki/Endogeneity_(econometrics%29). Maior número de filhos diminui oferta de trabalho, mas a situação de trabalho da mulher também tem efeitos no número de filhos. Portanto, a contribuição deste artigo foi a utilização de **variáveis instrumentais** para driblar este problema e estudar a *relação causal* entre fertilidade e oferta de trabalho.

As variaveis instrumentais utilizadas foram:
- **Dois primeiros filhos de mesmo sexo:**
Os autores se apoiam em um fenômeno amplamente observado de que pais de irmãos de mesmo sexo são significativamente mais propensos a ter mais um filho. Como a definição do sexo dos dois primeiros filhos é aleatória, temos um bom instrumento para avaliar o impacto do terceiro filho.
- **Gêmeos na segunda gravidez:**
Como a ocorrência de gravidez múltipla também é aleatória, esta é uma variável instrumental possível para estudar o impacto do terceiro filho na oferta de trabalho das mães.

Ambas as variáveis são boas pois além de aleatórias, não causam efeito direto na variável dependente que pretendemos analisar. Por exemplo, admitimos que relação causal existe apenas entre oferta de trabalho e terceiro filho, independentemente do sexo dos dois primeiros filhos ou de que o terceiro tenha vindo de uma gestação múltipla.

### Dados utilizados

Os autores extraíram parte de seus dados do censo de 1980 dos EUA.

Nós vamos utilizar os dados contidos em "census80.csv" que deriva do censo de 1980. Os dados são restritos às famílias com mães entre 21 e 35 anos, e que tem dois ou três filhos.

O arquivo contém as seguintes variáveis:
- workedm: se a mãe trabalha.
- weeksm: número de semanas que a mãe trabalha.
- whitem: mãe é branca.
- blackm: mãe é negra.
- hispm: mãe é hispânica.
- othracem: mãe é de outra raça.
- sex1st: sexo da primeiro filho (0 corresponde a masculino e 1 a feminino).
- sex2nd: sexo do segundo filho (0 corresponde a masculino 1 a feminino).
- ageq2nd: idade em trimestres (quarters) do segundo filho.
- ageq3rd: idade em trimestres (quarters) do terceiro filho.
- numberkids: número de crianças na família.

In [2]:
library(tidyverse)
library(AER)

df <- read_csv("census80.csv")

head(df)

Parsed with column specification:
cols(
  workedm = col_integer(),
  weeksm = col_integer(),
  whitem = col_integer(),
  blackm = col_integer(),
  hispm = col_integer(),
  othracem = col_integer(),
  sex1st = col_integer(),
  sex2nd = col_integer(),
  ageq2nd = col_integer(),
  ageq3rd = col_integer(),
  numberkids = col_integer()
)


workedm,weeksm,whitem,blackm,hispm,othracem,sex1st,sex2nd,ageq2nd,ageq3rd,numberkids
0,0,1,0,0,0,0,1,30,NA,2
1,52,1,0,0,0,0,1,9,NA,2
1,30,1,0,0,0,1,0,22,NA,2
0,0,1,0,0,0,0,1,12,NA,2
0,0,0,1,0,0,0,1,14,NA,2
0,0,1,0,0,0,0,0,50,48,3


In [3]:
summary(df)

    workedm           weeksm          whitem           blackm      
 Min.   :0.0000   Min.   : 0.00   Min.   :0.0000   Min.   :0.0000  
 1st Qu.:0.0000   1st Qu.: 0.00   1st Qu.:1.0000   1st Qu.:0.0000  
 Median :1.0000   Median :12.00   Median :1.0000   Median :0.0000  
 Mean   :0.5716   Mean   :20.82   Mean   :0.8314   Mean   :0.1125  
 3rd Qu.:1.0000   3rd Qu.:48.00   3rd Qu.:1.0000   3rd Qu.:0.0000  
 Max.   :1.0000   Max.   :52.00   Max.   :1.0000   Max.   :1.0000  
                                                                   
     hispm            othracem           sex1st           sex2nd      
 Min.   :0.00000   Min.   :0.00000   Min.   :0.0000   Min.   :0.0000  
 1st Qu.:0.00000   1st Qu.:0.00000   1st Qu.:0.0000   1st Qu.:0.0000  
 Median :0.00000   Median :0.00000   Median :0.0000   Median :0.0000  
 Mean   :0.02725   Mean   :0.02886   Mean   :0.4871   Mean   :0.4881  
 3rd Qu.:0.00000   3rd Qu.:0.00000   3rd Qu.:1.0000   3rd Qu.:1.0000  
 Max.   :1.00000   Max.   :1.0

Na amostra utilizada, contendo 427418 famílias, vemos que 57% das mães trabalham e 29% tem três filhos. Como esperado, os sexos dos filhos estão igualmente representados (49% mulheres).

A partir destes dados, vamos criar mais três variáveis indicadoras (*dummys*):
- multiple: variável indicadora para segunda gravidez múltipla (constatada se ageq2nd = ageq3rd)
- samesex: variável indicadora para dois primeiros filhos de mesmo sexo (constatado se sex1st = sex2nd)
- threekids: variável indicadora para três filhos (constatado se numberkids = 3)

"threekids" é o regressor em que estamos interessados, i.e. o fato de a família mudar de *dois* para *três* filhos.

As **variáveis instrumentais** "multiple" e "samesex" serão utilizadas para avaliar o impacto de "threekids".

In [4]:
df <- mutate(df, multiple = if_else(ageq2nd == ageq3rd, 1, 0, missing = 0)) %>%
  mutate(samesex = if_else(sex1st == sex2nd, 1, 0)) %>%
  mutate(threekids = if_else(numberkids == 3, 1, 0))

summary(df[12:14])

    multiple          samesex         threekids     
 Min.   :0.00000   Min.   :0.0000   Min.   :0.0000  
 1st Qu.:0.00000   1st Qu.:0.0000   1st Qu.:0.0000  
 Median :0.00000   Median :1.0000   Median :0.0000  
 Mean   :0.00729   Mean   :0.5019   Mean   :0.2861  
 3rd Qu.:0.00000   3rd Qu.:1.0000   3rd Qu.:1.0000  
 Max.   :1.00000   Max.   :1.0000   Max.   :1.0000  

Na amostra avaliada, composta por mães com dois ou três filhos, 0,7% tiveram uma segunda gravidez múltipla (gêmeos).

O resultado obtido (~50%) para a proporção de famílias com primeiro e segundo filho com mesmo sexo é o esperado em uma amostra aleatória. Isto porque a probabilidade do segundo filho ter determinado sexo *independe* do sexo do primeiro filho. Representando os resultados possíveis em relação ao sexo com $\{$*sexo do 1º filho, sexo do 2º filho*$\}$ o espaço amostral é dado por $\{\{M,M\},\{M,F\},\{F,M\},\{F,F\}\}$, e temos:
$$P(\{M,M\}\cup\{F,F\})=P(\{M,M\})+P(\{F,F\})=P(M)\cdot P(M)+P(F)\cdot P(F)=\frac{1}{2}\cdot\frac{1}{2}+\frac{1}{2}\cdot\frac{1}{2}=\frac{1}{2}$$
ou seja, a probabilidade da mãe ter dois filhos de mesmo sexo é de 50%.

### First Stage Regression
Para que uma variável instrumental possa ser utilizada, alguma correlação deve existir com a variável explicativa (*explanatory variable, regressor*) de interesse. Para observar a relação entre "samesex" e "threekids", faremos uma regressão de primeiro estágio:

$$threekids = \beta_{0} + \beta_{1} samesex + \epsilon$$

Em seguida, vamos avaliar um modelo que leva em conta possíveis diferenças devido a raça da mãe como efeito fixo. Ou seja, teremos  uma regressão multivariável, mas considerando que as diferenças entre raças são apenas ajustes na "linha base", e o efeito do tratamento ($\beta_1$) é fixo para as categorias.

$$threekids = \beta_{0} + \beta_{1} samesex + \beta_{2} blackm + \beta_{3} hispm + \beta_{4} othracem + \epsilon$$

In [5]:
fsr0 <- lm(threekids ~ samesex, df)
summary(fsr0)


Call:
lm(formula = threekids ~ samesex, data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-0.3105 -0.3105 -0.2615  0.6895  0.7385 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 0.261518   0.000978  267.40   <2e-16 ***
samesex     0.048991   0.001380   35.49   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.4513 on 427416 degrees of freedom
Multiple R-squared:  0.002938,	Adjusted R-squared:  0.002935 
F-statistic:  1259 on 1 and 427416 DF,  p-value: < 2.2e-16


Tomando a amostra como um todo, 26% das mães com dois filhos de sexos diferentes tiveram um terceiro filho ($\beta_{0}$). Esta porcentagem passa para 31% quando os dois primeiros filhos são de mesmo sexo ($\beta_{0}+\beta_{1}$). Há diferença significativa. Isto confirma este fenômeno já reportado, e mostra que é possível utilizar "samesex" como variável instrumental. 

In [6]:
# Regressão multivariável de primeiro estágio
fsr1 <- lm(threekids ~ samesex + blackm + hispm + othracem, df)
coeftest(fsr1)


t test of coefficients:

             Estimate Std. Error  t value  Pr(>|t|)    
(Intercept) 0.2515354  0.0010249 245.4174 < 2.2e-16 ***
samesex     0.0490182  0.0013786  35.5567 < 2.2e-16 ***
blackm      0.0659541  0.0021899  30.1171 < 2.2e-16 ***
hispm       0.0817364  0.0042432  19.2631 < 2.2e-16 ***
othracem    0.0111834  0.0041273   2.7096  0.006737 ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


Dois primeiros filhos de mesmo sexo aumentam a probabilidade de um terceiro em 4.9 pontos percentuais.

In [7]:
# Regressão multivariável de primeiro estágio, desta vez verificano o impacto de "multiple"
fsr2 <- lm(threekids ~ multiple + blackm + hispm + othracem, df)
coeftest(fsr2)


t test of coefficients:

             Estimate Std. Error  t value  Pr(>|t|)    
(Intercept) 0.2710109  0.0007523 360.2415 < 2.2e-16 ***
multiple    0.7179404  0.0080400  89.2965 < 2.2e-16 ***
blackm      0.0648870  0.0021730  29.8603 < 2.2e-16 ***
hispm       0.0817475  0.0042103  19.4159 < 2.2e-16 ***
othracem    0.0115414  0.0040954   2.8181  0.004831 ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


Para confirmar o óbvio impacto da gravidez múltipla no número de filhos, também realizamos uma regressão de primeiro estágio (fsr2). A proporção de três filhos nas mães com segunda gravidez múltipla obviamente é de 100%. Observando as estimativas da regressão, observamos alguns desvios, que ocorrem devido ao modelo "fixed effects" utilizado.

### Two Stage Least Squares Regression

A análise do efeito causal do terceiro filho na oferta de trabalho das mães prosseguirá utilizando o método de regressão de mínimos quadrados em dois estágios (two stage least squares). Intuitivamente, podemos entender o processo como se fossem dois estágios sequenciais:

- First Stage:  $3kids = \pi_{0} + \pi_{1}samesex + \omega$

- Second Stage:  $laborsupply = \beta_{0} + \beta_{1} \widehat{3kids} + \epsilon$

Os valores estimados no primeiro estágio ($\widehat{3kids}$) são introduzidos como variável explicativa no segundo estágio, e então $\hat{\beta_1}$ será a estimativa da relação causal entre "threekids" e "laborsupply".

Na prática, o tratamento é um pouco diferente para que as incertezas e variâncias sejam calculadas corretamente (cf. https://en.wikipedia.org/wiki/Instrumental_variables_estimation).

#### A) Utilizando "samesex" como instrumento

In [8]:
ivr1 <- ivreg(workedm ~ threekids + blackm + hispm + othracem
              | samesex + blackm + hispm + othracem, data = df)
coeftest(ivr1)


t test of coefficients:

              Estimate Std. Error t value  Pr(>|t|)    
(Intercept)  0.5857831  0.0085169 68.7792 < 2.2e-16 ***
threekids   -0.0982205  0.0306970 -3.1997  0.001376 ** 
blackm       0.1282025  0.0031330 40.9201 < 2.2e-16 ***
hispm       -0.0327583  0.0052636 -6.2235 4.865e-10 ***
othracem     0.0147760  0.0045175  3.2708  0.001073 ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [9]:
ivr2 <- ivreg(weeksm ~ threekids + blackm + hispm + othracem
              | samesex + blackm + hispm + othracem, data = df)
coeftest(ivr2)


t test of coefficients:

            Estimate Std. Error t value  Pr(>|t|)    
(Intercept) 21.42394    0.38029 56.3359 < 2.2e-16 ***
threekids   -4.99430    1.37066 -3.6437 0.0002688 ***
blackm       7.18208    0.13989 51.3400 < 2.2e-16 ***
hispm       -1.32648    0.23503 -5.6440 1.663e-08 ***
othracem     1.70933    0.20171  8.4740 < 2.2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


#### B) Utilizando "multiple" como instrumento

In [10]:
ivr3 <- ivreg(workedm ~ threekids + blackm + hispm + othracem
              | multiple + blackm + hispm + othracem, data = df)
coeftest(ivr3)


t test of coefficients:

              Estimate Std. Error  t value  Pr(>|t|)    
(Intercept)  0.5763680  0.0035006 164.6495 < 2.2e-16 ***
threekids   -0.0641256  0.0123194  -5.2053 1.938e-07 ***
blackm       0.1259529  0.0025249  49.8849 < 2.2e-16 ***
hispm       -0.0355364  0.0047392  -7.4984 6.474e-14 ***
othracem     0.0144008  0.0045073   3.1950  0.001398 ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [11]:
ivr4 <- ivreg(weeksm ~ threekids + blackm + hispm + othracem
              | multiple + blackm + hispm + othracem, data = df)
coeftest(ivr4)


t test of coefficients:

            Estimate Std. Error  t value  Pr(>|t|)    
(Intercept) 20.91123    0.15628 133.8082 < 2.2e-16 ***
threekids   -3.13765    0.54998  -5.7050 1.164e-08 ***
blackm       7.05958    0.11272  62.6299 < 2.2e-16 ***
hispm       -1.47777    0.21157  -6.9846 2.861e-12 ***
othracem     1.68889    0.20122   8.3932 < 2.2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


### Interpretação dos Resultados

Avaliando através de *"samesex"*, o terceiro filho causa uma diminuição de 10 pontos percentuais na proporção de mães que trabalham, e uma diminuição média de 5.0 semanas de trabalho.

Através de *"multiple"*, observamos que o terceiro filho causa uma diminuição de 6 pontos percentuais na proporção de mães que trabalham, e uma diminuição média de 3.1 semanas de trabalho.

Quando consideramos os erros padrão e intervalos de confiança razoáveis, vemos que a discrepância não é muito acentuada.

Entretanto, é importante notar que estamos lidando com um efeito **local**. A estimativa é o efeito médio local do tratamento (*local average treatment effect*). Ou seja, por meio de variáveis instrumentais estamos avaliando o efeito médio na oferta de trabalho devido ao terceiro filho concebido em função de: a) se ter irmãos de mesmo sexo ou b) a mãe ter uma segunda gravidez múltipla.

Em outras palavras, usando "samesex" avaliamos o efeito causal do terceiro filho na oferta de trabalho para mães cuja fertilidade foi afetada por terem dois primeiros filhos de mesmo sexo.
Já as estimativas via variável instrumental "multiple" refletem o efeito do terceiro filho na oferta de trabalho para aquelas mães que tiveram mais filhos devido a gravidez múltipla do que elas teriam de outra forma.
Deste modo, os instrumentos não necessariamente identificam os mesmos efeitos médios!

Angrist e Evans relatam que, como a probabilidade de gravidez de gêmeos não é exatamente igual entre grupos de mulheres, o uso da segunda gravidez múltipla pode gerar algumas distorções demográficas, por exemplo selecionando mulheres mais velhas (embora o efeito seja pequeno).

Além disso, dada a idade do segundo filho, os terceiros filhos nascidos de gravidez múltipla serão necessariamente mais velhos do que filhos nascidos em gravidez única, e a idade dos filhos tem impacto na oferta de trabalho das mães. Angrist e Evans analisam em maiores detalhes o efeito desta diferença de idade média dos filhos em função do instrumento utilizado, e demonstram que considerando este aspecto, as estimativas derivadas de "samesex" se aproximam das estimativas obtidas por meio de "multiple".

### Referências
- Joshua D. Angrist e William N. Evans, "Children and Their Parents' Labor Supply: Evidence from Exogenous Variation in Family Size", *The American Economic Review* Vol. 88, No. 3, 1998, pp. 450-477.

Esta análise foi baseada em um exercício proposto no curso "Data Analysis for Social Scientists" oferecido no segundo semestre de 2018 pelo MIT na plataforma EdX (MITx - 14.310x).